In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
from pprint import pprint

# Importing the models to be tested
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from auxiliar_func import *
from plot_func import *

In [2]:
df = pd.read_csv('Census-Income-KDD.csv')
target = 'income_50k'
df_tr, df_te = train_test_split(df, test_size=0.3, random_state=42)

TARGET_METRIC = 'f1_macro'
SEED = 42

prep_params_grid = {
    'scaling': [None, 'minmax', 'standard'],
    'imputation': ['mode'],
    'cat_age': [False, True],
    'merge_capital': [False, True],
    'downsampling_method': ['random', 'NearMiss'],
    'target_freq': [0.75, 0.8, 0.85],
    'generate_dummies': [True]
}

def n_comb(grid: dict, print_=True):
    n = 1
    for k in grid.keys():
        n *= len(grid[k])
    if print_:
        print(f'Number of model combinations to be tested: {n}')
    else:
        return n

def test_model(mod, prep_grid, mod_grid, name):
    if not os.path.exists(f'results_{name}.csv'):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore") # to avoid convergence warnings
                results = search_best_combination(mod, mod_grid, prep_grid, df_tr, target_metric=TARGET_METRIC)
                results.to_csv(f'results_{name}.csv', index=False)
        except Exception as e:
            print(e)
            return None
    
n_comb(prep_params_grid)

Number of model combinations to be tested: 72


### Tuning parameters for LDA

In [3]:
mod_par_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto'],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
}

n_comb(mod_par_grid)

lda = LDA()
test_model(lda, prep_params_grid, mod_par_grid, 'lda')

Number of model combinations to be tested: 36
===Iteration 1===
Searching preprocessing parameters...
it: 72/72
Searching model parameters...
it: 36/36
Best metric: 0.7581762636547423
Best preprocessing parameters: [{'scaling': 'standard', 'imputation': 'mode', 'cat_age': True, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': True, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': 'minmax', 'imputation': 'mode', 'cat_age': True, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': 'minmax', 'imputation': 'mode', 'cat_age': False, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': True, 'remove_duplicates': True}, {'scaling': N

### Tuning parameters for logistic regression

In [4]:
mod_par_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],
    'C': [0.1, 1, 10],
    'class_weight': [None, 'balanced'],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1, 10],
    'max_iter': [1000],
    'multi_class': ['auto'],
    'random_state': [SEED],
    'solver': ['saga'],
    'n_jobs': [-1],
}

n_comb(mod_par_grid)

logreg = LogisticRegression()
test_model(logreg, prep_params_grid, mod_par_grid, 'logreg')

Number of model combinations to be tested: 144
===Iteration 1===
Searching preprocessing parameters...


### Tuning parameters for SVM

In [ ]:
mod_par_grid = {
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge','hinge'],
    'dual': [False],
    'C': [0.1, 1, 10],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'random_state': [SEED]
}

n_comb(mod_par_grid)

svm = LinearSVC()
test_model(svm, prep_params_grid, mod_par_grid, 'svm')

### Tuning parameters for random forest

In [ ]:
mod_par_grid = {
    'n_estimators': [50, 75, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 20, 25, 30],
    'max_features': ['sqrt', 'log2'],
    'random_state': [SEED],
    'verbose': [0],
    'warm_start': [False],
    'class_weight': [None, 'balanced', 'balanced_subsample'],
    'n_jobs': [-1]
}

n_comb(mod_par_grid)

rf = RandomForestClassifier()
test_model(rf, prep_params_grid, mod_par_grid, 'rf')

### Tuning parameters for catboost

In [ ]:
prep_params_grid2 = {
    'scaling': [None],
    'imputation': ['mode'],
    'cat_age': [False],
    'merge_capital': [False, True],
    'downsampling_method': ['random', 'NearMiss'],
    'target_freq': [0.75, 0.8, 0.85],
    'generate_dummies': [False]
}

# a first preprocess to get the categorical features
df_tr_pre = preprocessing(df_tr, imputation='mode', cat_age=False, generate_dummies=False)
X_train, y_train = df_tr_pre.drop(target, axis=1), df_tr_pre[target]
cat_features = list(X_train.select_dtypes(include=['category']).columns)

mod_par_grid = {
    'iterations': [5],
    'depth': [6, 8, 10],
    'border_count': [32, 64, 128],
    'random_seed': [SEED],
    'verbose': [0],
    'loss_function': ['Logloss'],
    'eval_metric': ['F1', 'AUC'],
    'class_weights': [[1, 1], [1, 2], [1, 3]],
    'cat_features': [cat_features],
}

n_comb(mod_par_grid)

cat_model = CatBoostClassifier()
test_model(cat_model, prep_params_grid2, mod_par_grid, 'catboost')